In [1]:
!python -m pip install --user --upgrade pip

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


In [2]:
!pip3 install kfp --upgrade --user

In [3]:
import kfp
from kfp import dsl
import kfp.components as comp
from kfp.components import InputPath, OutputPath


In [4]:
output_dir ='/Users/user/Documents/kubeflow/kubeflow'

In [5]:
# Creating the content of the data loading script

def data_loading(data_path):
    '''
    Function for loading diabetes dataset
    '''
     
    import os
    import argparse
    import pandas as pd
    import pickle
    #reading the data from its source
    data = pd.read_csv("https://raw.githubusercontent.com/Youngprof3/kubeflow_test/main/project_on_diabetes_kubeflow/data_loading/dataset.csv")
     #Save the data as a pickle file to be used by the preprocess component.
    with open(f'{data_path}/working_data.pkl', 'wb') as f:
        pickle.dump(data, f)


In [6]:
data_loading(output_dir)

In [7]:
# Creating the content of the data preprocessing script



def data_preprocessing(data_path):
    
    import pandas as pd
    import argparse
    import numpy as np
    import pickle
    from sklearn.preprocessing import MinMaxScaler, Normalizer
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    
    with open(f'{data_path}/working_data.pkl', 'rb') as f:
        data = pickle.load(f)
        #data features
        X = data.iloc[:,:-1]
        #target data
        y = data.iloc[:,-1:]
        #encoding the categorical columns
        le = LabelEncoder()

        X['gender'] = le.fit_transform(X['gender'])
        y['class'] = le.fit_transform(y['class'])

        #splitting the data
        X_train,X_test,y_train,y_test = train_test_split( X,y, test_size=0.3, random_state = 42)
        #feature scaling
        ms =MinMaxScaler(feature_range=(0,1))
        X_train = ms.fit_transform(X_train)
        X_test = ms.transform(X_test)

        #Save the train_data as a pickle file to be used by the train component.
        with open(f'{data_path}/train_data.pkl', 'wb') as f:
            pickle.dump((X_train,  y_train), f)

        #Save the test_data as a pickle file to be used by the predict component.
        with open(f'{data_path}/test_data.pkl', 'wb') as f:
            pickle.dump((X_test,  y_test), f)





In [9]:
data_preprocessing(output_dir)

C:\Users\user\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [10]:
# Creating the content of the model training script


def model_training(data_path):
    
    import numpy as np
    import pickle
    import argparse
    import os
    from sklearn.svm import SVC
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier

    #loading the train data
    with open(f'{data_path}/train_data.pkl', 'rb') as f:
        train_data = pickle.load(f)
    #Separate the X_train from y_train.
        X_train, y_train = train_data

        log_regres = LogisticRegression(max_iter=10000)
        lr_model = log_regres.fit(X_train, y_train)

        svm = SVC(kernel ="linear", random_state=2)
        svm_model = svm.fit(X_train, y_train)

        knn = KNeighborsClassifier(n_neighbors=7)
        knn_model = knn.fit(X_train, y_train)


    with open(f'{data_path}/lr_model.pkl', 'wb') as f:
        pickle.dump(lr_model, f)
    
    with open(f'{data_path}/svm_model.pkl', 'wb') as f:
        pickle.dump(svm_model, f)
    
    with open(f'{data_path}/knn_model.pkl', 'wb') as f:
        pickle.dump(knn_model, f)
    

    


In [11]:
model_training(output_dir)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-10-653d7a35729f>:27: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn_model = knn.fit(X_train, y_train)


In [12]:
# Creating the content of the model testing script




def model_testing(data_path):
    
    import numpy as np
    import pickle
    import argparse
    import os
    #loading the test data
    
    with open(f'{data_path}/test_data.pkl', 'rb') as f:
        test_data = pickle.load(f)
    
    #Separate the X_test from y_test.
        X_test, y_test = test_data

    with open(f'{data_path}/lr_model.pkl', 'rb') as f:
        log_reg_model = pickle.load(f)
    
    with open(f'{data_path}/svm_model.pkl', 'rb') as f:
        svm_clas_model = pickle.load(f)
        
    with open(f'{data_path}/knn_model.pkl', 'rb') as f:
        knn_clas_model = pickle.load(f)
        
    

    y_pred_lr = log_reg_model.predict(X_test)
    y_pred_svm = svm_clas_model.predict(X_test)
    y_pred_knn = knn_clas_model.predict(X_test)

    
    with open(f'{data_path}/y_pred_lr.pkl', 'wb') as f:
        pickle.dump(y_pred_lr, f)
        
    with open(f'{data_path}/y_pred_svm.pkl', 'wb') as f:
        pickle.dump(y_pred_svm, f)
        
    with open(f'{data_path}/y_pred_knn.pkl', 'wb') as f:
        pickle.dump(y_pred_knn, f)
        
    




In [13]:
model_testing(output_dir)

In [14]:
# Creating the content of the model evaluation script



def model_evaluation(data_path):
    
    import numpy as np
    import pickle
    from sklearn.metrics import classification_report, confusion_matrix
    #loading the test data
    with open(f'{data_path}/test_data.pkl', 'rb') as f:
        test_data = pickle.load(f)
    #Separate the X_test from y_test.
        X_test, y_test = test_data

    with open(f'{data_path}/y_pred_lr.pkl', 'rb') as f:
         y_pred_lr = pickle.load(f)
            
    with open(f'{data_path}/y_pred_svm.pkl', 'rb') as f:
         y_pred_svm = pickle.load(f)
            
    with open(f'{data_path}/y_pred_knn.pkl', 'rb') as f:
         y_pred_knn = pickle.load(f)
    


    class_report_lr = classification_report(y_test, y_pred_lr)
    class_report_svm = classification_report(y_test, y_pred_svm)
    class_report_knn = classification_report(y_test, y_pred_knn)


    print(f'Classification report for the logistic regression model: {class_report_lr}')

    print(f'Classification report for the support vector machine model: {class_report_svm}')

    print(f'Classification report for the k-nearest neighbor model: {class_report_knn}')


    confusion_matrix_lr = confusion_matrix(y_test, y_pred_lr)
    confusion_matrix_svm = confusion_matrix(y_test, y_pred_svm)
    confusion_matrix_knn = confusion_matrix(y_test, y_pred_knn)

    print(f'Confusion matrix for the logistic regression model: {confusion_matrix_lr}')

    print(f'Confusion matrix for the support vector machine model: {confusion_matrix_svm}')

    print(f'Confusion matrix for the k-nearest neighbor model: {confusion_matrix_knn}')
    


In [15]:
model_evaluation(output_dir)

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report for the logistic regression model:               precision    recall  f1-score   support

           0       0.92      0.97      0.95        36
           2       1.00      0.10      0.18        10
           3       0.97      1.00      0.98       253
           4       0.00      0.00      0.00         1

    accuracy                           0.96       300
   macro avg       0.72      0.52      0.53       300
weighted avg       0.96      0.96      0.95       300

Classification report for the support vector machine model:               precision    recall  f1-score   support

           0       0.88      1.00      0.94        36
           2       0.67      0.20      0.31        10
           3       0.98      1.00      0.99       253
           4       0.00      0.00      0.00         1

    accuracy                           0.97       300
   macro avg       0.63      0.55      0.56       300
weighted avg       0.96      0.97      0.96       300

Classificatio

In [16]:
obtain_data_op = kfp.components.create_component_from_func(data_loading, base_image="python:3.10.4-slim-buster" ,packages_to_install=['pandas'])

preprocess_op = kfp.components.create_component_from_func(data_preprocessing, base_image="python:3.10.4-slim-buster"  ,packages_to_install=['pandas','sklearn'])

model_train_op = kfp.components.create_component_from_func(model_training, base_image="python:3.10.4-slim-buster"  ,packages_to_install=['pandas', 'sklearn'])

model_test_op = kfp.components.create_component_from_func(model_testing, base_image="python:3.10.4-slim-buster" ,packages_to_install=['pandas', 'sklearn'] )

model_evaluation_op = kfp.components.create_component_from_func(model_evaluation, base_image="python:3.10.4-slim-buster" ,packages_to_install=['pandas', 'sklearn'] )

In [17]:
data_path = '/Users/user/Documents/code/lightweight_component'

In [18]:
@dsl.pipeline(name="Dibetes Pipeline Lightweight", description="Performs preprocessing, training, testing and prediction if a patient is diabetic or not")

def diabetic_prediction():
    pvc_op = dsl.VolumeOp(name='Persistent Volume Claim', 
                resource_name='data-volume',
                size='2Gi',
                modes=dsl.VOLUME_MODE_RWO)

     #create obtain data component
    
    obtain_data_task = obtain_data_op(data_path).add_pvolumes({data_path: pvc_op.volume})
    # Create preprocess components.
   
    preprocess_task = preprocess_op(data_path).add_pvolumes({data_path: pvc_op.volume}).after(obtain_data_task)
    # Create train component.
    
    model_train_task = model_train_op(data_path).add_pvolumes({data_path: pvc_op.volume}).after(preprocess_task)
    # Create test component.
    
    model_test_task = model_test_op(data_path).add_pvolumes({data_path: pvc_op.volume}).after(model_train_task)
    # Create evaluation component.
    
    model_evaluation_task = model_evaluation_op(data_path).add_pvolumes({data_path: pvc_op.volume}).after(model_test_task)

    

kfp.compiler.Compiler().compile(diabetic_prediction, 'diabetic_prediction_pvc.yaml')